## modeling of estimation on sqft price

In [2]:
%load_ext autoreload
# always reload modules marked with "%aimport"
%autoreload 1

import os,sys
import numpy as np
import pandas as pd

# add the 'src' directory as one where we can import modules
src_dir = os.path.join(os.getcwd(), '../..', 'src')
sys.path.append(src_dir)

%aimport plot_utils
from plot_utils import *
%aimport preprocessing
from preprocessing import *
%aimport data_source
from data_source import *

import logging
from utils import *
set_logger(logging.DEBUG)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### DataSource 
* This module loads data from database
* Auto update views by executing database/preprocess.sql

In [3]:
ds = DataSource(host='localhost',port=5432, username='postgres', password='', database='sdra')
ds.show_views()
ds.update_views()

[dp] connect to database postgresql://postgres:@localhost:5432/sdra
[dp] supported views: ['properties', 'property_features', 'transactions', 'property_addresses', 'property_address_transactions', 'property_transaction_valid']
[dp] execute c:\wenyan\dse_capstone\database\preprocess.sql


### Preprocess
* This module will deliver the valid dataset with finished features

In [4]:
df_transaction = ds.get_view_df('property_address_transactions')
df_property = ds.get_view_df('property_addresses')
print(df_transaction.shape)
print(df_property.shape)

(1512560, 21)
(709325, 18)


In [5]:
pp = Preprocess(df_transaction, df_property)

### dataset for modeling
Preprocess.dataset(feature='delivered', valid=True)
* suggest using: feature='delivered', otherwise all features including non-solid features will be included
* suggest using: valid=True, otherwise outliers and invalid feature values will be  included
* date range can be specified

In [6]:
# records of all years
df_all_valid = pp.dataset(feature='delivered', valid=True)

[dp] transforming date
[dp] transforming view
[dp] transforming pool
[dp] transforming impr_over_land
[dp] transforming sold_year
[dp] transforming sold_age
[dp] making sqft_zip_avg
[dp] making sqft_price_zip_avg
[dp] making sold_price_zip_avg
[dp] making sale_count_zip
[dp] making prop_count_zip


In [7]:
# records of last year
df_last_year_valid = pp.dataset(feature='delivered', valid=True, date=('2016-10-01', '2017-10-01'))
df_last_year_valid.shape

[dp] transforming date
[dp] transforming view
[dp] transforming pool
[dp] transforming impr_over_land
[dp] transforming sold_year
[dp] transforming sold_age
[dp] making sqft_zip_avg
[dp] making sqft_price_zip_avg
[dp] making sold_price_zip_avg
[dp] making sale_count_zip
[dp] making prop_count_zip


(37630, 11)

In [8]:
df_last_year_valid[:3]

,sqft_price_zip_avg,impr_over_land,sqft_zip_avg,sold_price,date,num_bath,sold_price_zip_avg,pool,sqft,num_bed,view
0,333.554978,2.004593,2610.605195,717000,2017-09-27,2.0,864917.316017,0,2112,4.0,0
1,333.554978,0.313287,2610.605195,673000,2017-09-27,2.0,864917.316017,0,1419,3.0,0
2,333.554978,2.445162,2610.605195,720000,2017-09-25,2.0,864917.316017,0,1819,3.0,0


In [13]:
# records of last 2 years
df_last_2year_valid = pp.dataset(feature='delivered', valid=True, date=('2015-10-01', '2017-10-01'))
df_last_2year_valid.shape

transforming view
transforming pool
transforming date
transforming year_built
transforming sold_year
transforming sold_age
making sqft_zip_avg
making sqft_price_zip_avg
making sold_price_zip_avg
making sale_count_zip
making prop_count_zip


(76689, 9)

In [14]:
# records of last 3 years
df_last_3year_valid = pp.dataset(feature='delivered', valid=True, date=('2014-10-01', '2017-10-01'))
df_last_3year_valid.shape

transforming view
transforming pool
transforming date
transforming year_built
transforming sold_year
transforming sold_age
making sqft_zip_avg
making sqft_price_zip_avg
making sold_price_zip_avg
making sale_count_zip
making prop_count_zip


(111380, 9)

### regression model
* target: sqft_price
* try various features combination: ...
* try different dataset size: last 1 year, 2 years, ...
* try different models: LinearRegression, RandomForestRegression, ...